<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo I: Construyendo y visualizando frecuencias de palabras

En este apartado, nos centraremos en la función auxiliar `build_freqs()` y en la visualización de un conjunto de datos introducido en ella. En nuestro objetivo de análisis de sentimiento de tweets, esta función construirá un diccionario donde podemos buscar cuántas veces aparece una palabra en las listas de tweets positivos o negativos. 

Iniciamos importando las librerías

#### Importaremos algunas funciones auxiliares que proporcionamos en el archivo utils.py:

* `process_tweet()`: Limpia el texto, lo convierte en token en palabras separadas, elimina las palabras vacías y convierte las palabras en raíces.
* `build_freqs()`: Cuenta la frecuencia con la que una palabra del "corpus" (el conjunto completo de tweets) se asoció con una etiqueta positiva "1" o una etiqueta negativa "0". Luego construye el diccionario `freqs`, donde cada clave es una tupla` (palabra, etiqueta) `, y el valor es el recuento de su frecuencia dentro del corpus de tweets.

## Cargar el conjunto de datos de muestra NLTK

Utilizaremos la [base de datos de tweeter de NLTK](http://www.nltk.org/howto/twitter.html#Using-a-Tweet-Corpus).

A continuación, crearemos una matriz de etiquetas que coincida con los sentimientos de nuestros tweets. Este tipo de datos funciona de forma muy similar a una lista normal, pero está optimizado para cálculos y manipulación. La matriz `labels` estará compuesta por 10000 elementos. Los primeros 5000 se llenarán con etiquetas "1" que denotan sentimientos positivos, y los siguientes 5000 serán etiquetas "0" que denotan lo contrario. Podemos hacer esto fácilmente con una serie de operaciones proporcionadas por la biblioteca `numpy`:

* `np.ones()` 
* `np.zeros()` 
* `np.append()` - concatena arrays

## Recordatorio de uso de dicionarios

### Accessing values and lookup keys

Performing dictionary lookups and retrieval are common tasks in NLP. There are two ways to do this: 

* Using square bracket notation: This form is allowed if the lookup key is in the dictionary. It produces an error otherwise.
* Using the [get()](https://docs.python.org/3/library/stdtypes.html#dict.get) method: This allows us to set a default value if the dictionary key does not exist. 

Let us see these in action:

When using a square bracket lookup, it is common to use an if-else block to check for containment first (with the keyword `in`) before getting the item. On the other hand, you can use the `.get()` method if you want to set a default value when the key is not found. Let's compare these in the cells below:

## Diccionario de frecuencia de palabras

Echemos un vistazo a la función **build_freqs()** en **utils.py**. Esta es la función que crea el diccionario que contiene los recuentos de palabras de cada corpus.

```python
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1    
    return freqs
```

Como se muestra arriba, cada clave es una tupla de 2 elementos que contiene un par "(palabra, y)". La "palabra" es un elemento en un tweet procesado, mientras que "y" es un número entero que representa el corpus: "1" para los tweets positivos y "0" para los tweets negativos. El valor asociado con esta clave es el número de veces que esa palabra aparece en el corpus especificado. Por ejemplo:

```
# "folowfriday" aparece 25 veces en los tweets positivos
('sigue el viernes', 1.0): 25

# "shame" aparece 19 veces en los tweets negativos
('shame', 0.0): 19
```

Ahora es el momento de usar el diccionario devuelto por la función `build_freqs()`. Primero, alimentemos nuestras listas de `tweets` y` etiquetas` y luego imprimamos un informe básico:

Desafortunadamente, esto no ayuda mucho a comprender los datos. Sería mejor visualizar este resultado para obtener mejores conocimientos.

## Tabla de recuentos de palabras

Seleccionaremos un conjunto de palabras que nos gustaría visualizar. Es mejor almacenar esta información temporal en una tabla que sea muy fácil de usar más adelante.

In [1]:
#seleccione algunas palabras para que aparezcan en el informe. asumiremos que cada palabra es única (es decir, sin duplicados)

keys = ['happi', 'merri', 'nice', 'good', 'bad', 'sad', 'mad', 'best', 'pretti',
        '❤', ':)', ':(', '😒', '😬', '😄', '😍', '♛',
        'song', 'idea', 'power', 'play', 'magnific']


Luego podemos usar un diagrama de dispersión para inspeccionar esta tabla visualmente. En lugar de graficar los conteos sin procesar, lo trazaremos en la escala logarítmica para tener en cuenta las amplias discrepancias entre los conteos sin procesar (por ejemplo, `:)` tiene 3568 conteos en positivo mientras que solo 2 en negativo). La línea roja marca el límite entre las áreas positivas y negativas. Las palabras cercanas a la línea roja se pueden clasificar como neutrales.

Este cuadro es sencillo de interpretar. Muestra que los emoticones `:)` y `: (` son muy importantes para el análisis de sentimientos. Por lo tanto, ¡no debemos permitir que los pasos de preprocesamiento eliminen estos símbolos!

¿Qué pasa con el símbolo de la corona?